### Загружаем отобранные id моделей на яндекс маркете.

###### вопрос отбора id моделей остается открытый, самы простой способ запросить через api топ популярных моделей на яндекс.маркете

In [74]:
with open("model_ids_2.txt") as file:
    model_ids = [row.strip() for row in file]

In [39]:
import pandas as pd

### Класс для работы с Яднекс.маркет API

Создаем класс для работы с API яндекс маркета
первая функция генерирует запрос с учетом переданной модели
вторая генерирует сам запрос и возвращает json ответ с отзывами, на вход получает id модели и набор параметром: количество отзывов на одной странице; номер страницы с отзывами; и т. д.,
третья функция просто обертка для удобства.

In [56]:
import requests
import json


class YandexMarketContent(object):
    
  class NotAuthorized(BaseException):
    pass

  def __init__(self, key=None):
    if not key:
      raise YandexMarketContent.NotAuthorized("You must provide authorization key to access Yandex.Market API!")
    self.key = key

  def _make_opinions_url(self, model, format='json'):
    return 'https://api.content.market.yandex.ru/v2/models/'+ model +'/opinions/'

  def _make_request(self, model, params):
    url = self._make_opinions_url(model)
    query_params = params
    params['Authorization'] = self.key

    response = requests.get(url, params=params, headers={'Authorization':self.key, 'Accept':'*/*'})
    output = json.loads(response.content)
    if response.status_code == 401:
      server_response = []
      for error in output['errors']:
        server_response.append(error)
      raise YandexMarketContent.NotAuthorized("Your key `%s' wasn't authorized at Yandex.Market API. Server response: %s" % (self.key, server_response))

    return output

  def req(self, model, params):
    columns = ['opinion','grade']
    df_temp = pd.DataFrame(columns=columns)
    response = self._make_request(model, params)
    for i in range(0,30):
        op = ''
        grade = ''
        if ('opinions' in response):
            if  ('cons' in response['opinions'][i]):
                op += response['opinions'][i]['cons'] + " "
            if ('pros' in response['opinions'][i]):
                 op += response['opinions'][i]['pros'] + " "
            if ('text' in response['opinions'][i]):
                 op += response['opinions'][i]['text']
        else:
            continue
        grade = response['opinions'][i]['grade']
        df_temp = df_temp.append({'opinion': op, 'grade': grade}, ignore_index=True)
    return df_temp

Нас интересуют поля cons, pros, text и grade
из полей cons, pros и text формируем отзыв, а grade - оценка по пятибальной шкале

### Формирование dataset'a отзывов

создаем объект с полученным api ключом

In [71]:
api = YandexMarketContent(key='8e85dd1c-4bc5-45ec-a877-5932f0246826')

Проходим по трем страницам отзывов каждой модели, на каждой странице 30 отзывов
Получаем по 90 отзывов на каждый id модели и сохраняем их в массив для дальнейшей обработки

###### Чтобы не превышать лимит Яндекса в 10 запросов в секунду, сделаем таймер по одному запросу каждую секунду

In [ ]:
columns = ['opinion','grade']
df = pd.DataFrame(columns=columns)

In [75]:
%%time
import time
for i in range(1,2):
    for model in model_ids:
        df = df.append(api.req(model, params={'count':30, 'page':i}), ignore_index=True)
        time.sleep(1)

CPU times: user 3.45 s, sys: 79 ms, total: 3.53 s
Wall time: 1min 15s


#### Посмотрим размер получившегося массива

In [77]:
len(df)

4830

Сохраним получившейся набор

In [80]:
df.to_pickle('data_op.pkl')

In [93]:
df = pd.read_pickle('data_op.pkl')

 все оценки 4 и 5 делаем положительными (1) 
оценки 3,2,1 - отрицательные (0) 
и удаляем все лишние столбцы

In [97]:
df.loc[df['grade'] < 4, 'grade'] = 0
df.loc[df['grade'] > 3, 'grade'] = 1

In [98]:
df['grade']

0       1
1       1
2       1
3       1
4       1
5       1
6       1
7       1
8       1
9       1
10      1
11      1
12      1
13      0
14      1
15      1
16      1
17      1
18      1
19      1
20      0
21      0
22      1
23      0
24      0
25      1
26      0
27      0
28      0
29      1
       ..
4800    0
4801    1
4802    0
4803    0
4804    0
4805    0
4806    0
4807    1
4808    0
4809    1
4810    1
4811    1
4812    0
4813    0
4814    0
4815    0
4816    0
4817    0
4818    1
4819    0
4820    0
4821    1
4822    0
4823    1
4824    1
4825    0
4826    0
4827    1
4828    1
4829    1
Name: grade, Length: 4830, dtype: object

Теперь приступаем к работе с датафреймом

In [99]:
df.head()

,opinion,grade
0,1. Нет NFC\n2. Даже 32 ГБ внутренней памяти эт...,1
1,"Расположение датчика отпечатка пальца Батарея,...",1
2,- сяоми забросили эту модель и присылают тольк...,1
3,Камера оставляет желать лучшего.\n По соотноше...,1
4,Автономность через год активного использования...,1


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4830 entries, 0 to 4829
Data columns (total 2 columns):
opinion    4830 non-null object
grade      4830 non-null object
dtypes: object(2)
memory usage: 75.5+ KB


в данных есть пропуски, удалим их

In [101]:
df = df.dropna()

посмотрим разделение по классам, как видно отрицательных отзывов почти в два раза меньше положительных

In [102]:
df.groupby(['grade']).count()

,opinion
grade,
0,1658
1,3172


Подготовим данные для обучения

In [112]:
X = df['opinion'].values
y = df['grade']
y=y.astype('int')

In [113]:
y

0       1
1       1
2       1
3       1
4       1
5       1
6       1
7       1
8       1
9       1
10      1
11      1
12      1
13      0
14      1
15      1
16      1
17      1
18      1
19      1
20      0
21      0
22      1
23      0
24      0
25      1
26      0
27      0
28      0
29      1
       ..
4800    0
4801    1
4802    0
4803    0
4804    0
4805    0
4806    0
4807    1
4808    0
4809    1
4810    1
4811    1
4812    0
4813    0
4814    0
4815    0
4816    0
4817    0
4818    1
4819    0
4820    0
4821    1
4822    0
4823    1
4824    1
4825    0
4826    0
4827    1
4828    1
4829    1
Name: grade, Length: 4830, dtype: int64

## Тестируем модели

попробуем "в лоб" линейные модели

In [104]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

In [114]:
model = Pipeline([
    ('vect', CountVectorizer()),
    ('lr', LogisticRegression()),
])
score = cross_val_score(model, X, y, cv=5)
print('Accuracy: ', score.mean())
print('Score std: ', score.std())

Accuracy:  0.8105530527666372
Score std:  0.014339362628069455


Точность: 0.8140897458121502

In [115]:
model_2 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('lr', LogisticRegression()),
])
score_2 = cross_val_score(model_2, X, y, cv=5)
print('Score mean: ', score_2.mean())
print('Score std: ', score_2.std())

Score mean:  0.8031032755116152
Score std:  0.006596645293570202


Точность: 0.797933829336682

In [116]:
model_3 = Pipeline([
    ('vect', CountVectorizer(min_df=10)),
    ('lr', LogisticRegression()),
])
score_3 = cross_val_score(model_3, X, y, cv=5)
print('Score mean: ', score_3.mean())
print('Score std: ', score_3.std())

Score mean:  0.8033094553237229
Score std:  0.01799713670299746


Точность: 0.766345924592172

In [117]:
model_4 = Pipeline([
    ('vect', CountVectorizer(min_df=50)),
    ('lr', LogisticRegression()),
])
score_4 = cross_val_score(model_4, X, y, cv=5)
print('Score mean: ', score_4.mean())
print('Score std: ', score_4.std())

Score mean:  0.7697594115503857
Score std:  0.01895462929628957


Точность: 0.789472853618645

In [118]:
model_5 = Pipeline([
    ('vect', CountVectorizer()),
    ('LSVC', LinearSVC()),
])
score_5 = cross_val_score(model_5, X, y, cv=5)
print('Score mean: ', score_5.mean())
print('Score std: ', score_5.std())



Score mean:  0.7881831403167506
Score std:  0.025756548582510863


Точность: 0.752814298547248

In [119]:
model_6 = Pipeline([
    ('vect', CountVectorizer()),
    ('SGD', SGDClassifier()),
])
score_6 = cross_val_score(model_6, X, y, cv=5)
print('Score mean: ', score_6.mean())
print('Score std: ', score_6.std())

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/s

Score mean:  0.788816904268077
Score std:  0.012356022543668169


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Точность: 0.797933829336682

Попробуем с биграммами

In [120]:
model_7 = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1,2))),
    ('lr', LogisticRegression()),
])
score_7 = cross_val_score(model_7, X, y, cv=5)
print('Accuracy with bigrams: ', score_7.mean())

Accuracy with bigrams:  0.8180111898709839


Точность: 0.825089392841651

In [121]:
model_8 = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1,3))),
    ('lr', LogisticRegression()),
])
score_8 = cross_val_score(model_8, X, y, cv=5)
print('Accuracy with bigrams: ', score_8.mean())

Accuracy with bigrams:  0.814077868006124


Точность: 0.772957183450843

 ## GridSearch для формирования финальной модели

выберем из двух наиболее точных пайплайнов с помощью gridsearch

In [122]:
pipe = Pipeline([('vectorizer', CountVectorizer()), ('classifier', LogisticRegression())])

In [123]:
param_grid = [
    {'classifier': [LogisticRegression(), SGDClassifier()], 
    'vectorizer': [CountVectorizer(), TfidfVectorizer()],
    'vectorizer__min_df': [1, 2, 10, 50],
    'vectorizer__ngram_range': [(1,1), (1,2), (1,3)],
    'vectorizer__analyzer':['word', 'char_wb']},
    {'classifier': [LogisticRegression(), SGDClassifier()], 
    'vectorizer': [CountVectorizer(), TfidfVectorizer()],
    'vectorizer__min_df': [1, 2, 10, 50],
    'vectorizer__ngram_range': [(3,5)],
    'vectorizer__analyzer':['char_wb']}
]

In [124]:
grid = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)

In [ ]:
%%time
grid.fit(X, y)
print(grid.best_params_)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/s

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/s

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/s

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/s

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/s

...

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/s

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/s

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/s

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/s

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/s

Наилучший результат у SGDClassifier + TfidfVectorizer с биграммами
##### Точность: 0.8459127079816735

In [ ]:
print(grid.best_score_)

Попробуем подобрать альфа у классификатора

In [ ]:
param_grid = [
    {'classifier': [SGDClassifier(class_weight='balanced')], 
    'vectorizer': [TfidfVectorizer(ngram_range=(1,2))],
    'classifier__alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}]

In [ ]:
grid = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)

In [ ]:
%%time
grid.fit(X, y)
print(grid.best_params_)

получились стандартные параметры и такая же точность

In [ ]:
print(grid.best_score_)

## Предсказываем и записываем в файл

Разбираем файл с тестовым набором

In [ ]:
with open('test.csv', 'rU') as f:
    data = f.read()
test_op = [r.text for r in bs4.BeautifulSoup(data, 'lxml').find_all('review')]

Предсказываем значения и записываем в файл

In [ ]:
out = pd.DataFrame(grid.predict(test_op), columns=['y'])
out.index.name = 'Id'
out['y'] = out['y'].apply(lambda x: 'neg' if (x==0) else 'pos')
out.to_csv('subm.csv')